1. Khai báo thư viện

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter

# Kiểm tra và tải nếu gói chưa được cài đặt
def download_nltk_package(package_name):
    try:
        nltk.data.find(f'tokenizers/{package_name}')
    except LookupError:
        nltk.download(package_name)

# Kiểm tra và tải các gói cần thiết
download_nltk_package('stopwords')
download_nltk_package('punkt')
download_nltk_package('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HaoPham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HaoPham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2. Load data

In [2]:
# Tải dữ liệu để xử  lí
path = r"D:\DATN\Social Media Sentiments Analysis Dataset\New_version\dataset\synthetic_social_media_data.csv"
data = pd.read_csv(path)
df =  pd.DataFrame(data)
df.head()

,Post ID,Post Content,Sentiment Label,Number of Likes,Number of Shares,Number of Comments,User Follower Count,Post Date and Time,Post Type,Language
0,aa391375-7355-44b7-bcbf-97fb4e5a2ba3,Word who nor center everything better politica...,Neutral,157,243,64,4921,2024-01-10 00:14:21,video,fr
1,1c9ec98d-437a-48d9-9cba-bd5ad853c59a,Begin administration population good president...,Positive,166,49,121,612,2024-02-03 00:20:11,image,es
2,170e5b5b-1d9a-4d02-a957-93c4dbb18908,Thousand total sign. Agree product relationshi...,Positive,185,224,179,9441,2024-07-25 14:20:23,video,de
3,aec53496-60ee-4a06-8821-093a04dc8770,Individual from news third. Oil forget them di...,Neutral,851,369,39,6251,2024-02-20 09:15:09,text,de
4,4eacddb7-990d-4056-8784-7e1d5c4d1404,Time adult letter see reduce. Attention sudden...,Negative,709,356,52,1285,2024-03-01 04:17:35,image,de


3. Kiểm tra dữ liệu

In [3]:
# Thông tin về dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Post ID              2000 non-null   object
 1   Post Content         2000 non-null   object
 2   Sentiment Label      2000 non-null   object
 3   Number of Likes      2000 non-null   int64 
 4   Number of Shares     2000 non-null   int64 
 5   Number of Comments   2000 non-null   int64 
 6   User Follower Count  2000 non-null   int64 
 7   Post Date and Time   2000 non-null   object
 8   Post Type            2000 non-null   object
 9   Language             2000 non-null   object
dtypes: int64(4), object(6)
memory usage: 156.4+ KB


In [4]:
# Thống kê cơ bản
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Number of Likes,2000.0,503.379,283.210866,1.0,267.00,505.0,740.00,1000.0
Number of Shares,2000.0,248.485,145.094990,0.0,121.00,248.5,375.00,500.0
Number of Comments,2000.0,102.805,58.032886,0.0,52.00,104.5,153.25,200.0
User Follower Count,2000.0,5005.777,2913.865254,3.0,2383.25,5048.5,7556.50,9999.0


In [5]:
# Kiểm tra dữ liệu null
df.isnull().sum()

Post ID                0
Post Content           0
Sentiment Label        0
Number of Likes        0
Number of Shares       0
Number of Comments     0
User Follower Count    0
Post Date and Time     0
Post Type              0
Language               0
dtype: int64

In [6]:
# Kiểm tra dữ liệu trùng lặp
df.duplicated().sum()

np.int64(0)

In [7]:
# Tổng quan 
df.shape

(2000, 10)

Sau khi kiểm tra dữ liệu thì có nhưng điểm chính như sau:

    + Tổng quang: có 2000 dòng và 10 cột
    + Gía trị thiếu: không có giá trị thiếu
    + Các trường dữ liệu: trong đó chỉ có 1 trường là phân loại: 'Post Date and Time'(theo dạng ngày),
    còn lại hậu như là theo dạng số và chuỗi.
    + Ngoại lệ: không có các ngoại lệ
    + Các dữ liệu trùng lập: không có dữ liệu trùng lập
-----------------------------------------------------------------
Nhận xét: Dữ liệu hiện đang có 3 cột không cần thiêt là 'Language','User Follower Count', 'Post ID' để đi đúng vào vấn đề cần phân tích thì sẽ không chọn. Cũng như dữ liệu trong mỗi dòng ở cột 'Post Content' có chứa các từ gây cản trở quá trình phân tích cần xử lí. Sau cùng là định dạng dữ liệu ở cột 'Post Date and Time' lại thành Datetime.

4. Xử lý dữ liệu

4.1 Xử lý các trường dữ liệu

In [8]:
# Chọn lại các cột cần thiết trong quá trình phân tích
df_selected = df[['Post Content','Sentiment Label','Number of Likes','Number of Shares','Number of Comments','Post Date and Time','Post Type']]
df_selected.head()

,Post Content,Sentiment Label,Number of Likes,Number of Shares,Number of Comments,Post Date and Time,Post Type
0,Word who nor center everything better politica...,Neutral,157,243,64,2024-01-10 00:14:21,video
1,Begin administration population good president...,Positive,166,49,121,2024-02-03 00:20:11,image
2,Thousand total sign. Agree product relationshi...,Positive,185,224,179,2024-07-25 14:20:23,video
3,Individual from news third. Oil forget them di...,Neutral,851,369,39,2024-02-20 09:15:09,text
4,Time adult letter see reduce. Attention sudden...,Negative,709,356,52,2024-03-01 04:17:35,image


In [9]:
# Hàm xử lý dữ liệu trong cột "Post Content"
def preprocess_text_column(df, column_name):
    # Khởi tạo lemmatizer và danh sách stopwords
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def clean_text(text):
        """Làm sạch văn bản: loại bỏ ký tự đặc biệt, emoji và chữ số."""
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Chỉ giữ lại chữ cái
        text = re.sub(r'\s+', ' ', text).strip()  # Xóa khoảng trắng thừa
        return text

    def to_lower(text):
        """Chuyển văn bản về chữ thường."""
        return text.lower()

    def tokenize(text):
        """Tokenize văn bản thành các từ."""
        return word_tokenize(text)

    def get_custom_stopwords():
        """Tạo danh sách stopwords tùy chỉnh từ cột 'Post Content'."""
        all_posts = ' '.join(df[column_name].astype(str))  # Kết hợp tất cả nội dung
        tokens_all = word_tokenize(all_posts)  # Tokenize tất cả nội dung
        word_counts = Counter(tokens_all)  # Tính tần suất xuất hiện của các từ

        # Lấy các từ không có nghĩa (có thể điều chỉnh điều kiện này)
        user_stop_words = {word for word, count in word_counts.items() if count > 2 and len(word) < 5}
        
        # Kết hợp với stopwords mặc định
        return stop_words.union(user_stop_words)

    def remove_stopwords(tokens, custom_stop_words):
        """Loại bỏ các stopwords khỏi danh sách các từ."""
        return [word for word in tokens if word not in custom_stop_words]

    def lemmatize_tokens(tokens):
        """Thực hiện lemmatization cho các từ."""
        return [lemmatizer.lemmatize(word) for word in tokens]

    def preprocess_text(text, custom_stop_words):
        """Tiền xử lý văn bản: gọi các hàm riêng biệt."""
        text = to_lower(text)
        text = clean_text(text)
        tokens = tokenize(text)
        tokens = remove_stopwords(tokens, custom_stop_words)
        tokens = lemmatize_tokens(tokens)
        return ' '.join(tokens)

    # Tạo danh sách stopwords tùy chỉnh từ cột dữ liệu
    custom_stop_words = get_custom_stopwords()

    # Áp dụng tiền xử lý cho toàn bộ cột và lưu vào cột mới "Processed Content"
    df['Processed Content'] = df[column_name].apply(lambda x: preprocess_text(x, custom_stop_words))

    return df

# Ví dụ sử dụng để xử lý cột "Post Content"
# Gọi hàm preprocess_text_column để xử lý cột "Post Content"
df_selected = preprocess_text_column(df_selected, "Post Content")
df_selected.head()

C:\Users\HaoPham\AppData\Local\Temp\ipykernel_7136\876819217.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Processed Content'] = df[column_name].apply(lambda x: preprocess_text(x, custom_stop_words))


,Post Content,Sentiment Label,Number of Likes,Number of Shares,Number of Comments,Post Date and Time,Post Type,Processed Content
0,Word who nor center everything better politica...,Neutral,157,243,64,2024-01-10 00:14:21,video,center everything better political various cou...
1,Begin administration population good president...,Positive,166,49,121,2024-02-03 00:20:11,image,begin administration population president part...
2,Thousand total sign. Agree product relationshi...,Positive,185,224,179,2024-07-25 14:20:23,video,thousand total agree product relationship seve...
3,Individual from news third. Oil forget them di...,Neutral,851,369,39,2024-02-20 09:15:09,text,individual third forget different account whit...
4,Time adult letter see reduce. Attention sudden...,Negative,709,356,52,2024-03-01 04:17:35,image,adult letter reduce attention suddenly agency ...


Xóa cột Post content cũ  thay  thế bằng cột đã xử lí 

In [10]:
# Xóa cột "Post Content" gốc
df_selected = df_selected.drop(columns=['Post Content'])

# Đổi tên cột "Processed Content" thành "Post Content"
df_selected.rename(columns={'Processed Content': 'Post Content'}, inplace=True)

post_content = df_selected.pop('Post Content')
df_selected.insert(0, 'Post Content', post_content)

# Kết quả cuối cùng
df_selected.head() # Hiển thị các dòng đầu tiên của DataFrame

,Post Content,Sentiment Label,Number of Likes,Number of Shares,Number of Comments,Post Date and Time,Post Type
0,center everything better political various cou...,Neutral,157,243,64,2024-01-10 00:14:21,video
1,begin administration population president part...,Positive,166,49,121,2024-02-03 00:20:11,image
2,thousand total agree product relationship seve...,Positive,185,224,179,2024-07-25 14:20:23,video
3,individual third forget different account whit...,Neutral,851,369,39,2024-02-20 09:15:09,text
4,adult letter reduce attention suddenly agency ...,Negative,709,356,52,2024-03-01 04:17:35,image


4.2 Chuẩn hóa dữ liệu

In [11]:
# Đổi định dạng cột Timestamp thành kiểu datetime
df_selected['Post Date and Time'] = pd.to_datetime(df_selected['Post Date and Time'])
# Đổi định dạng cột Likes, Retweets thành kiểu int64
df_selected[['Number of Likes', 'Number of Shares', 'Number of Comments']] = df_selected[['Number of Likes', 'Number of Shares', 'Number of Comments']].astype('int64')

4.3 Dữ liệu sau khi xử lí

In [13]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Post Content        2000 non-null   object        
 1   Sentiment Label     2000 non-null   object        
 2   Number of Likes     2000 non-null   int64         
 3   Number of Shares    2000 non-null   int64         
 4   Number of Comments  2000 non-null   int64         
 5   Post Date and Time  2000 non-null   datetime64[ns]
 6   Post Type           2000 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 109.5+ KB


In [15]:
df_selected.shape

(2000, 7)

5. Lưu file đã xử lí

In [ ]:
df_selected.to_csv(r"D:\DATN\Social Media Sentiments Analysis Dataset\New_version\dataset\procced\synthetic_social_media_data_procced.csv", index=False)
